In [1]:
""" IMPORT NEEDED MODULES """

import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import cmocean
import cmocean.cm as cmo

import torch
import torch.utils.data

import re
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

import time as t
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data.dataset import Dataset  # For custom datasets

In [2]:
startingDir = os.getcwd()
print(startingDir)

C:\Users\Peter\Desktop\Thermo_R_Code\FinalThermo\new\BCI-ML


In [15]:
"""
# Takes in a string that represents name of csv file and 
# converts that string to one that matches the date string present
# in the old master file (old master file refers to the raw excel file
# filled with extraneous data from BCI)that contains the date/times 
# for recorded GPP values
Description pulled from make-master-non-c0ontinuous

# takes a file name in the form YYYY_MM_DD_HH_MM_bci_t and returns new date in form 2/17/2015 15:30. 
# with times not at 30 minute intervals rounded to the nearest 30 min

"""

# takes a file name in the form YYYY_MM_DD_HH_MM_bci_t and returns new date in form 2/17/2015 15:30. 
# with times not at 30 minute intervals rounded to the nearest 30 min
def getRoundedTime(inTime):
    minDivFiveFlag = True
    year = inTime[2:4]
    month = inTime[5:7]
    day = inTime[8:10]
    hour = inTime[11:13]
    minuteInt = int(inTime[14:16])
    # Round minute such that those 0-15 and 45-59 get rounded to 0 and all other values to 30
    minuteStr = "XX"
    if(minuteInt <= 15):
        minuteStr = "00"
    elif(minuteInt >= 45):
        minuteStr = "00"
        hour = str(int(hour)+1)
    else:
        minuteStr = "30"
    second = "00"
    if(int(hour) == 0):
        hour = "0"
    elif(int(hour) < 10):
        hour = hour[1:2]
    # No leading zeroes in day or month
    if(int(month) < 10):
        month = month[1:2]
    if(int(day) < 10):
        day = day[1:2]
    result = month + "/" + day + "/" + year + " " + hour + ":" + minuteStr #+ ":" + second + " " + timeDay
    return result

In [25]:
""" Takes in a date in form 2/17/2015 15:30 and another date in the same form, checks if times are the same"""
def timeMatch(inTimeOne, inTimeTwo):
    timeOne = inTimeOne[inTimeOne.index(" "):]
    timeTwo = inTimeTwo[inTimeTwo.index(" "):]
    if(timeOne == timeTwo):
        return True
    else:
        return False

print(timeMatch("9/30/2315 15:00", "9/30/15 15:00"))

True


In [26]:
csvfilesMLLocation = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new2/csvdataFULL/BCIML_Dataset"
masterfileMLLocation = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new2/csvdataFULL/BCIML_Dataset/populatedGPPsFULL.csv"
testData = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new/BCI-ML/Monthly_Data_Summaries_csv/DAYTIME_new9930834___/test_data_DAYTIME_13-446994955314361.csv"
analysisLocation = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new2/tempplots"
all_working = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new2/all-30min-Apr2018_working.csv"

# Get data from test data, where we store targets and predictions of network
testDataInfo = pd.read_csv(testData)
testDataFileNames = testDataInfo['filename']
allInfo = pd.read_csv(all_working)
par_vals = allInfo['PAR']
date_vals = allInfo['date']

In [27]:
# Each data frame holds PAR and corresponding date values, each split into individual months
#
par_date_dic = {"PAR": par_vals, "date": date_vals}
full_df = pd.concat(par_date_dic,axis=1)
regex_time_range = re.compile("2015_.._.._(0[4-9]_..|1[0-9]_..|20_00)_bci_t")

# Filter out time space we want ####################################
#with contains and regex abo e

print(full_df)


#https://stackoverflow.com/questions/27975069/how-to-filter-rows-containing-a-string-pattern-from-a-pandas-dataframe
feb_df = full_df[full_df['date'].str.startswith('2/')]
mar_df = full_df[full_df['date'].str.startswith('3/')]
apr_df = full_df[full_df['date'].str.startswith('4/')]
may_df = full_df[full_df['date'].str.startswith('5/')]
jun_df = full_df[full_df['date'].str.startswith('6/')]
jul_df = full_df[full_df['date'].str.startswith('7/')]
aug_df = full_df[full_df['date'].str.startswith('8/')]
sep_df = full_df[full_df['date'].str.startswith('9/')]

# save data frames
par_csv_save_location = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new/BCI-ML/Images and Plots/PAR Data"
os.chdir(par_csv_save_location)
feb_df.to_csv('feb_par.csv')
mar_df.to_csv('mar_par.csv')
apr_df.to_csv('apr_par.csv')
may_df.to_csv('may_par.csv')
jun_df.to_csv('jun_par.csv')
jul_df.to_csv('jul_par.csv')
aug_df.to_csv('aug_par.csv')
sep_df.to_csv('sep_par.csv')
full_df.to_csv('full_par.csv')

              PAR           date
0     2043.833333  2/17/15 13:00
1     1013.350000  2/17/15 13:30
2      804.966667  2/17/15 14:00
3      658.200000  2/17/15 14:30
4      424.350000  2/17/15 15:00
...           ...            ...
8363     0.340000  9/30/15 21:30
8364     0.340000  9/30/15 22:00
8365     0.340167  9/30/15 22:30
8366     0.340000  9/30/15 23:00
8367     0.340000  9/30/15 23:30

[8368 rows x 2 columns]


PermissionError: [Errno 13] Permission denied: 'full_par.csv'

In [28]:
"""Goes through a list of filenames, rounding their time values to the nearest 30 min interval,
and then looking at PAR values from the mastermaster file (all-30min-Apr2018_working) for the 
same times but +/- 6 days apart (THIS CAN BE CHANGED, MARKED WITH #XKDCD.) 
The average, median, standard deviation and range is then pulled form these
14 days of PAR data. Each of these values is saved, along with the exact time PAR value, and the 
difference between these two data points"""

# os.chdir(par_csv_save_location)
# sourceFile = "populatedGPPsDAYTIME.csv"
# daytimePopulatedInfo =pd.read_csv(sourceFile)
# filenames = daytimePopulatedInfo["FILE"]
## SELF CROSSING
os.chdir(par_csv_save_location)
sourceFile = "full_par.csv"
daytimePopulatedInfo =pd.read_csv(sourceFile)
filenames = daytimePopulatedInfo["date"]

savedIndex = 0
# Holds individual rows of table, will be concatted
fullInfoList = list()
# resultCsvFileName = "PAR_Analysis_" + testData[testData.rfind('/')+1:testData.rfind('.')] + ".csv"
# resultTextFileName = "Par_Analysis_textlog_" + testData[testData.rfind('/')+1:testData.rfind('.')] +".txt"
resultCsvFileName = "PAR_full_analysis" + testData[testData.rfind('/')+1:testData.rfind('.')] + ".csv"
resultTextFileName = "Par_Analysis_full_textlog_" + testData[testData.rfind('/')+1:testData.rfind('.')] +".txt"


savedIndex = 0
for filename in filenames:
    #newFilename = getRoundedTime(filename)
    newFilename = filename
    for i, r in full_df.iloc[savedIndex:].iterrows():
        # Go through each row where last left off, looking for matching time
        if(r['date']==newFilename):
            print(newFilename)
            savedIndex = i
            statList = list()
            backIdx = i-1
            forwardIdx = i+1
            statList.append(full_df.iloc[i]["PAR"])
            #XCKD, can change range of days pulled from here by changing size of list
            # where (size of list - 1) / 2 = number of days in each direction
            while(len(statList) < 15):
                # Start iterating backwards and forwards, adding PAR values to stat list as we go
                # foward and backindex will stop if they reach endpoints, and the other will grow the list
                # if both endpoints are reached, we stop.
                if(backIdx > -1):
                    if(timeMatch(newFilename, full_df.iloc[backIdx]['date'])):
                        statList.append(full_df.iloc[backIdx]['PAR'])
                if(forwardIdx < len(full_df)):
                    if(timeMatch(newFilename, full_df.iloc[forwardIdx]['date'])):
                        statList.append(full_df.iloc[forwardIdx]['PAR'])
                backIdx = backIdx - 1
                forwardIdx = forwardIdx + 1
                if(forwardIdx >= len(full_df) and backIdx <= -1):
                    break
            # If we can find no similar times to take average PAR from then log this in text file and move on
            if(len(statList) <= 1):
                with open(resultTextFileName, 'a') as out:
                    out.write(newFilename + ' has no additional par to take average of...')
                    break
            statSeries = pd.Series(statList)
            descSeries = statSeries.describe()
            detailsSeries = pd.Series({'newFilename': newFilename,'PAR': full_df.iloc[i]["PAR"], 'PARDIFF':abs(full_df.iloc[i]["PAR"]-descSeries['mean'])})
            finalInfoSeries = pd.concat([detailsSeries, descSeries])
            fullInfoList.append(finalInfoSeries)
            break

newSeries = (pd.concat(fullInfoList, axis=1)).transpose()
print("ASDASDASD")
print(newSeries)
print(type(newSeries))
newSeries.to_csv(par_csv_save_location+"/"+resultCsvFileName)
        

2/17/15 13:00
2/17/15 13:30
2/17/15 14:00
2/17/15 14:30
2/17/15 15:00
2/17/15 15:30
2/17/15 16:00
2/17/15 16:30
2/17/15 17:00
2/17/15 17:30
2/17/15 18:00
2/17/15 18:30
2/17/15 19:00
2/17/15 19:30
2/17/15 20:00
2/17/15 20:30
2/17/15 21:00
2/17/15 21:30
2/17/15 22:00
2/17/15 22:30
2/17/15 23:00
2/17/15 23:30
2/18/15 0:00
2/18/15 0:30
2/18/15 1:00
2/18/15 1:30
2/18/15 2:00
2/18/15 2:30
2/18/15 3:00
2/18/15 3:30
2/18/15 4:00
2/18/15 4:30
2/18/15 5:00
2/18/15 5:30
2/18/15 6:00
2/18/15 6:30
2/18/15 7:00
2/18/15 7:30
2/18/15 8:00
2/18/15 8:30
2/18/15 9:00
2/18/15 9:30
2/18/15 10:00
2/18/15 10:30
2/18/15 11:00
2/18/15 11:30
2/18/15 12:00
2/18/15 12:30
2/18/15 13:00
2/18/15 13:30
2/18/15 14:00
2/18/15 14:30
2/18/15 15:00
2/18/15 15:30
2/18/15 16:00
2/18/15 16:30
2/18/15 17:00
2/18/15 17:30
2/18/15 18:00
2/18/15 18:30
2/18/15 19:00
2/18/15 19:30
2/18/15 20:00
2/18/15 20:30
2/18/15 21:00
2/18/15 21:30
2/18/15 22:00
2/18/15 22:30
2/18/15 23:00
2/18/15 23:30
2/19/15 0:00
2/19/15 0:30
2/19/15 1:00
2

3/2/15 4:30
3/2/15 5:00
3/2/15 5:30
3/2/15 6:00
3/2/15 6:30
3/2/15 7:00
3/2/15 7:30
3/2/15 8:00
3/2/15 8:30
3/2/15 9:00
3/2/15 9:30
3/2/15 10:00
3/2/15 10:30
3/2/15 11:00
3/2/15 11:30
3/2/15 12:00
3/2/15 12:30
3/2/15 13:00
3/2/15 13:30
3/2/15 14:00
3/2/15 14:30
3/2/15 15:00
3/2/15 15:30
3/2/15 16:00
3/2/15 16:30
3/2/15 17:00
3/2/15 17:30
3/2/15 18:00
3/2/15 18:30
3/2/15 19:00
3/2/15 19:30
3/2/15 20:00
3/2/15 20:30
3/2/15 21:00
3/2/15 21:30
3/2/15 22:00
3/2/15 22:30
3/2/15 23:00
3/2/15 23:30
3/3/15 0:00
3/3/15 0:30
3/3/15 1:00
3/3/15 1:30
3/3/15 2:00
3/3/15 2:30
3/3/15 3:00
3/3/15 3:30
3/3/15 4:00
3/3/15 4:30
3/3/15 5:00
3/3/15 5:30
3/3/15 6:00
3/3/15 6:30
3/3/15 7:00
3/3/15 7:30
3/3/15 8:00
3/3/15 8:30
3/3/15 9:00
3/3/15 9:30
3/3/15 10:00
3/3/15 10:30
3/3/15 11:00
3/3/15 11:30
3/3/15 12:00
3/3/15 12:30
3/3/15 13:00
3/3/15 13:30
3/3/15 14:00
3/3/15 14:30
3/3/15 15:00
3/3/15 15:30
3/3/15 16:00
3/3/15 16:30
3/3/15 17:00
3/3/15 17:30
3/3/15 18:00
3/3/15 18:30
3/3/15 19:00
3/3/15 19:30
3/3/

3/15/15 8:00
3/15/15 8:30
3/15/15 9:00
3/15/15 9:30
3/15/15 10:00
3/15/15 10:30
3/15/15 11:00
3/15/15 11:30
3/15/15 12:00
3/15/15 12:30
3/15/15 13:00
3/15/15 13:30
3/15/15 14:00
3/15/15 14:30
3/15/15 15:00
3/15/15 15:30
3/15/15 16:00
3/15/15 16:30
3/15/15 17:00
3/15/15 17:30
3/15/15 18:00
3/15/15 18:30
3/15/15 19:00
3/15/15 19:30
3/15/15 20:00
3/15/15 20:30
3/15/15 21:00
3/15/15 21:30
3/15/15 22:00
3/15/15 22:30
3/15/15 23:00
3/15/15 23:30
3/16/15 0:00
3/16/15 0:30
3/16/15 1:00
3/16/15 1:30
3/16/15 2:00
3/16/15 2:30
3/16/15 3:00
3/16/15 3:30
3/16/15 4:00
3/16/15 4:30
3/16/15 5:00
3/16/15 5:30
3/16/15 6:00
3/16/15 6:30
3/16/15 7:00
3/16/15 7:30
3/16/15 8:00
3/16/15 8:30
3/16/15 9:00
3/16/15 9:30
3/16/15 10:00
3/16/15 10:30
3/16/15 11:00
3/16/15 11:30
3/16/15 12:00
3/16/15 12:30
3/16/15 13:00
3/16/15 13:30
3/16/15 14:00
3/16/15 14:30
3/16/15 15:00
3/16/15 15:30
3/16/15 16:00
3/16/15 16:30
3/16/15 17:00
3/16/15 17:30
3/16/15 18:00
3/16/15 18:30
3/16/15 19:00
3/16/15 19:30
3/16/15 20:00
3/

3/27/15 21:00
3/27/15 21:30
3/27/15 22:00
3/27/15 22:30
3/27/15 23:00
3/27/15 23:30
3/28/15 0:00
3/28/15 0:30
3/28/15 1:00
3/28/15 1:30
3/28/15 2:00
3/28/15 2:30
3/28/15 3:00
3/28/15 3:30
3/28/15 4:00
3/28/15 4:30
3/28/15 5:00
3/28/15 5:30
3/28/15 6:00
3/28/15 6:30
3/28/15 7:00
3/28/15 7:30
3/28/15 8:00
3/28/15 8:30
3/28/15 9:00
3/28/15 9:30
3/28/15 10:00
3/28/15 10:30
3/28/15 11:00
3/28/15 11:30
3/28/15 12:00
3/28/15 12:30
3/28/15 13:00
3/28/15 13:30
3/28/15 14:00
3/28/15 14:30
3/28/15 15:00
3/28/15 15:30
3/28/15 16:00
3/28/15 16:30
3/28/15 17:00
3/28/15 17:30
3/28/15 18:00
3/28/15 18:30
3/28/15 19:00
3/28/15 19:30
3/28/15 20:00
3/28/15 20:30
3/28/15 21:00
3/28/15 21:30
3/28/15 22:00
3/28/15 22:30
3/28/15 23:00
3/28/15 23:30
3/29/15 0:00
3/29/15 0:30
3/29/15 1:00
3/29/15 1:30
3/29/15 2:00
3/29/15 2:30
3/29/15 3:00
3/29/15 3:30
3/29/15 4:00
3/29/15 4:30
3/29/15 5:00
3/29/15 5:30
3/29/15 6:00
3/29/15 6:30
3/29/15 7:00
3/29/15 7:30
3/29/15 8:00
3/29/15 8:30
3/29/15 9:00
3/29/15 9:30
3/29

4/10/15 3:30
4/10/15 4:00
4/10/15 4:30
4/10/15 5:00
4/10/15 5:30
4/10/15 6:00
4/10/15 6:30
4/10/15 7:00
4/10/15 7:30
4/10/15 8:00
4/10/15 8:30
4/10/15 9:00
4/10/15 9:30
4/10/15 10:00
4/10/15 10:30
4/10/15 11:00
4/10/15 11:30
4/10/15 12:00
4/10/15 12:30
4/10/15 13:00
4/10/15 13:30
4/10/15 14:00
4/10/15 14:30
4/10/15 15:00
4/10/15 15:30
4/10/15 16:00
4/10/15 16:30
4/10/15 17:00
4/10/15 17:30
4/10/15 18:00
4/10/15 18:30
4/10/15 19:00
4/10/15 19:30
4/10/15 20:00
4/10/15 20:30
4/10/15 21:00
4/10/15 21:30
4/10/15 22:00
4/10/15 22:30
4/10/15 23:00
4/10/15 23:30
4/11/15 0:00
4/11/15 0:30
4/11/15 1:00
4/11/15 1:30
4/11/15 2:00
4/11/15 2:30
4/11/15 3:00
4/11/15 3:30
4/11/15 4:00
4/11/15 4:30
4/11/15 5:00
4/11/15 5:30
4/11/15 6:00
4/11/15 6:30
4/11/15 7:00
4/11/15 7:30
4/11/15 8:00
4/11/15 8:30
4/11/15 9:00
4/11/15 9:30
4/11/15 10:00
4/11/15 10:30
4/11/15 11:00
4/11/15 11:30
4/11/15 12:00
4/11/15 12:30
4/11/15 13:00
4/11/15 13:30
4/11/15 14:00
4/11/15 14:30
4/11/15 15:00
4/11/15 15:30
4/11/15 16:

...

6/21/15 0:30
6/21/15 1:00
6/21/15 1:30
6/21/15 2:00
6/21/15 2:30
6/21/15 3:00
6/21/15 3:30
6/21/15 4:00
6/21/15 4:30
6/21/15 5:00
6/21/15 5:30
6/21/15 6:00
6/21/15 6:30
6/21/15 7:00
6/21/15 7:30
6/21/15 8:00
6/21/15 8:30
6/21/15 9:00
6/21/15 9:30
6/21/15 10:00
6/21/15 10:30
6/21/15 11:00
6/21/15 11:30
6/21/15 12:00
6/21/15 12:30
6/21/15 13:00
6/21/15 13:30
6/21/15 14:00
6/21/15 14:30
6/21/15 15:00
6/21/15 15:30
6/21/15 16:00
6/21/15 16:30
6/21/15 17:00
6/21/15 17:30
6/21/15 18:00
6/21/15 18:30
6/21/15 19:00
6/21/15 19:30
6/21/15 20:00
6/21/15 20:30
6/21/15 21:00
6/21/15 21:30
6/21/15 22:00
6/21/15 22:30
6/21/15 23:00
6/21/15 23:30
6/22/15 0:00
6/22/15 0:30
6/22/15 1:00
6/22/15 1:30
6/22/15 2:00
6/22/15 2:30
6/22/15 3:00
6/22/15 3:30
6/22/15 4:00
6/22/15 4:30
6/22/15 5:00
6/22/15 5:30
6/22/15 6:00
6/22/15 6:30
6/22/15 7:00
6/22/15 7:30
6/22/15 8:00
6/22/15 8:30
6/22/15 9:00
6/22/15 9:30
6/22/15 10:00
6/22/15 10:30
6/22/15 11:00
6/22/15 11:30
6/22/15 12:00
6/22/15 12:30
6/22/15 13:00
6/2

7/3/15 19:00
7/3/15 19:30
7/3/15 20:00
7/3/15 20:30
7/3/15 21:00
7/3/15 21:30
7/3/15 22:00
7/3/15 22:30
7/3/15 23:00
7/3/15 23:30
7/4/15 0:00
7/4/15 0:30
7/4/15 1:00
7/4/15 1:30
7/4/15 2:00
7/4/15 2:30
7/4/15 3:00
7/4/15 3:30
7/4/15 4:00
7/4/15 4:30
7/4/15 5:00
7/4/15 5:30
7/4/15 6:00
7/4/15 6:30
7/4/15 7:00
7/4/15 7:30
7/4/15 8:00
7/4/15 8:30
7/4/15 9:00
7/4/15 9:30
7/4/15 10:00
7/4/15 10:30
7/4/15 11:00
7/4/15 11:30
7/4/15 12:00
7/4/15 12:30
7/4/15 13:00
7/4/15 13:30
7/4/15 14:00
7/4/15 14:30
7/4/15 15:00
7/4/15 15:30
7/4/15 16:00
7/4/15 16:30
7/4/15 17:00
7/4/15 17:30
7/4/15 18:00
7/4/15 18:30
7/4/15 19:00
7/4/15 19:30
7/4/15 20:00
7/4/15 20:30
7/4/15 21:00
7/4/15 21:30
7/4/15 22:00
7/4/15 22:30
7/4/15 23:00
7/4/15 23:30
7/5/15 0:00
7/5/15 0:30
7/5/15 1:00
7/5/15 1:30
7/5/15 2:00
7/5/15 2:30
7/5/15 3:00
7/5/15 3:30
7/5/15 4:00
7/5/15 4:30
7/5/15 5:00
7/5/15 5:30
7/5/15 6:00
7/5/15 6:30
7/5/15 7:00
7/5/15 7:30
7/5/15 8:00
7/5/15 8:30
7/5/15 9:00
7/5/15 9:30
7/5/15 10:00
7/5/15 10:30


8/23/15 6:30
8/23/15 7:00
8/23/15 7:30
8/23/15 8:00
8/23/15 8:30
8/23/15 9:00
8/23/15 9:30
8/23/15 10:00
8/23/15 10:30
8/23/15 11:00
8/23/15 11:30
8/23/15 12:00
8/23/15 12:30
8/23/15 13:00
8/23/15 13:30
8/23/15 14:00
8/23/15 14:30
8/23/15 15:00
8/23/15 15:30
8/23/15 16:00
8/23/15 16:30
8/23/15 17:00
8/23/15 17:30
8/23/15 18:00
8/23/15 18:30
8/23/15 19:00
8/23/15 19:30
8/23/15 20:00
8/23/15 20:30
8/23/15 21:00
8/23/15 21:30
8/23/15 22:00
8/23/15 22:30
8/23/15 23:00
8/23/15 23:30
8/24/15 0:00
8/24/15 0:30
8/24/15 1:00
8/24/15 1:30
8/24/15 2:00
8/24/15 2:30
8/24/15 3:00
8/24/15 3:30
8/24/15 4:00
8/24/15 4:30
8/24/15 5:00
8/24/15 5:30
8/24/15 6:00
8/24/15 6:30
8/24/15 7:00
8/24/15 7:30
8/24/15 8:00
8/24/15 8:30
8/24/15 9:00
8/24/15 9:30
8/24/15 10:00
8/24/15 10:30
8/24/15 11:00
8/24/15 11:30
8/24/15 12:00
8/24/15 12:30
8/24/15 13:00
8/24/15 13:30
8/24/15 14:00
8/24/15 14:30
8/24/15 15:00
8/24/15 15:30
8/24/15 16:00
8/24/15 16:30
8/24/15 17:00
8/24/15 17:30
8/24/15 18:00
8/24/15 18:30
8/24/

9/5/15 3:00
9/5/15 3:30
9/5/15 4:00
9/5/15 4:30
9/5/15 5:00
9/5/15 5:30
9/5/15 6:00
9/5/15 6:30
9/5/15 7:00
9/5/15 7:30
9/5/15 8:00
9/5/15 8:30
9/5/15 9:00
9/5/15 9:30
9/5/15 10:00
9/5/15 10:30
9/5/15 11:00
9/5/15 11:30
9/5/15 12:00
9/5/15 12:30
9/5/15 13:00
9/5/15 13:30
9/5/15 14:00
9/5/15 14:30
9/5/15 15:00
9/5/15 15:30
9/5/15 16:00
9/5/15 16:30
9/5/15 17:00
9/5/15 17:30
9/5/15 18:00
9/5/15 18:30
9/5/15 19:00
9/5/15 19:30
9/5/15 20:00
9/5/15 20:30
9/5/15 21:00
9/5/15 21:30
9/5/15 22:00
9/5/15 22:30
9/5/15 23:00
9/5/15 23:30
9/6/15 0:00
9/6/15 0:30
9/6/15 1:00
9/6/15 1:30
9/6/15 2:00
9/6/15 2:30
9/6/15 3:00
9/6/15 3:30
9/6/15 4:00
9/6/15 4:30
9/6/15 5:00
9/6/15 5:30
9/6/15 6:00
9/6/15 6:30
9/6/15 7:00
9/6/15 7:30
9/6/15 8:00
9/6/15 8:30
9/6/15 9:00
9/6/15 9:30
9/6/15 10:00
9/6/15 10:30
9/6/15 11:00
9/6/15 11:30
9/6/15 12:00
9/6/15 12:30
9/6/15 13:00
9/6/15 13:30
9/6/15 14:00
9/6/15 14:30
9/6/15 15:00
9/6/15 15:30
9/6/15 16:00
9/6/15 16:30
9/6/15 17:00
9/6/15 17:30
9/6/15 18:00
9/6/15 

9/23/15 15:00
9/23/15 15:30
9/23/15 16:00
9/23/15 16:30
9/23/15 17:00
9/23/15 17:30
9/23/15 18:00
9/23/15 18:30
9/23/15 19:00
9/23/15 19:30
9/23/15 20:00
9/23/15 20:30
9/23/15 21:00
9/23/15 21:30
9/23/15 22:00
9/23/15 22:30
9/23/15 23:00
9/23/15 23:30
9/24/15 0:00
9/24/15 0:30
9/24/15 1:00
9/24/15 1:30
9/24/15 2:00
9/24/15 2:30
9/24/15 3:00
9/24/15 3:30
9/24/15 4:00
9/24/15 4:30
9/24/15 5:00
9/24/15 5:30
9/24/15 6:00
9/24/15 6:30
9/24/15 7:00
9/24/15 7:30
9/24/15 8:00
9/24/15 8:30
9/24/15 9:00
9/24/15 9:30
9/24/15 10:00
9/24/15 10:30
9/24/15 11:00
9/24/15 11:30
9/24/15 12:00
9/24/15 12:30
9/24/15 13:00
9/24/15 13:30
9/24/15 14:00
9/24/15 14:30
9/24/15 15:00
9/24/15 15:30
9/24/15 16:00
9/24/15 16:30
9/24/15 17:00
9/24/15 17:30
9/24/15 18:00
9/24/15 18:30
9/24/15 19:00
9/24/15 19:30
9/24/15 20:00
9/24/15 20:30
9/24/15 21:00
9/24/15 21:30
9/24/15 22:00
9/24/15 22:30
9/24/15 23:00
9/24/15 23:30
9/25/15 0:00
9/25/15 0:30
9/25/15 1:00
9/25/15 1:30
9/25/15 2:00
9/25/15 2:30
9/25/15 3:00
9/25/

In [31]:
targetFileLocation = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new/BCI-ML/Monthly_Data_Summaries_csv/DAYTIME_new9930834___"
os.chdir(targetFileLocation)
test_analysis_csv = pd.read_csv('analysis_test_data_DAYTIME_13-446994955314361.csv')
os.chdir(csvfilesMLLocation)
par_analysis_csv = pd.read_csv('C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new/BCI-ML/Images and Plots/PAR Data/PAR_Analysis_test_data_DAYTIME_13-446994955314361.csv')

test_analysis_csv['filename'] = test_analysis_csv['filename'].apply(getRoundedTime)
new_merged_df = test_analysis_csv.merge(par_analysis_csv, left_on='filename',right_on='newFilename')
new_merged_df = new_merged_df.drop_duplicates(subset=['targets','outputs','filename'])
print(new_merged_df)
new_merged_df.to_csv(par_csv_save_location+"/"+"merged2_PAR_Analysis_test_data_DAYTIME_13-446994955314361.csv")

         targets    outputs       filename    gppdiff  Unnamed: 0  \
0       7.599151  27.038311  9/21/15 10:30  19.439160       24634   
5       8.349766  17.324461  9/21/15 10:30   8.974695       24634   
10     27.154482   8.614139  5/27/15 14:00  18.540343       12852   
17      0.000000  18.391531  5/30/15 12:00  18.391531       13359   
24     33.215084  14.834977   9/6/15 12:30  18.380107       22969   
...          ...        ...            ...        ...         ...   
22188  13.801845  13.809522   4/5/15 14:00   0.007677        3897   
22195  27.686996  27.694389  6/17/15 10:00   0.007393       15264   
22202   9.553397   9.546285  6/25/15 17:30   0.007113       16771   
22207   4.517934   4.512564  8/26/15 14:00   0.005370       21039   
22214   2.978327   2.979312   9/21/15 6:00   0.000986       24591   

         newFilename          PAR     PARDIFF  count         mean         std  \
0      9/21/15 10:30   327.833333  845.625641   13.0  1173.458974  514.017027   
5      9/